In [1]:
from __future__ import division

In [2]:
import os
import random
import math
import datetime
import time

In [3]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [5]:
features = pd.read_csv('data/random/features_100.csv', names=range(42), dtype='float32')
targets = pd.read_csv('data/random/targets_100.csv', names=['win', 'lose', 'draw'], dtype='float32')

num_hold_out = 50000

shuff = list(features.index)
random.shuffle(shuff)

holdout_features = features.loc[shuff[:num_hold_out]]
holdout_targets = targets.loc[shuff[:num_hold_out]]

features = features.loc[shuff[num_hold_out:]]
targets = targets.loc[shuff[num_hold_out:]]

In [6]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
178537,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51121,-1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,...,-1.0,-1.0,1.0,0.0,1.0,-1.0,1.0,-1.0,0.0,0.0
626406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1313851,-1.0,1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0
1003753,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
targets.head()

,win,lose,draw
178537,1.0,0.0,0.0
51121,1.0,0.0,0.0
626406,0.0,0.0,1.0
1313851,0.0,1.0,0.0
1003753,0.0,1.0,0.0


In [8]:
len(features)

2121908

In [9]:
def get_batch_iter(batch_size, batch_idx, dfs):
    
    length = len(dfs[0])
    for df in dfs:
        assert len(df) == length
        
    batches_per_df = int(math.ceil(length / batch_size))
    
    local_idx = batch_idx % batches_per_df
    
    start = local_idx*batch_size
    end = (local_idx+1)*batch_size
    
    return [df.iloc[start:end] for df in dfs]


def get_batch_random(batch_size, _, dfs):    
    mask = pd.Series(dfs[0].index).sample(n=batch_size, replace=False)    
    return [df.loc[mask] for df in dfs]


def get_batch(batch_size, batch_idx, dfs, how='iter'):
    if how == 'iter':
        return get_batch_iter(batch_size, batch_idx, dfs)
    elif how == 'random':
        return get_batch_random(batch_size, batch_idx, dfs)
    else:
        raise Exception()

In [10]:
def run(graph, output_prefix, batch_size, num_batches, batch_how='iter', save_prefix=None):
    
    with tf.Session(graph=graph) as sess:
    
        K.set_session(sess)
        sess.run(init_op)
    
        suffix = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
        name = '{}_{}'.format(output_prefix, suffix)
        train_writer = tf.summary.FileWriter(name, sess.graph)
        
        print "Running {}".format(name)
            
        t = time.time()
        delta_t = 0
        
        for i in range(num_batches):
        
            if i % 1000 == 0:
                
                delta_t = time.time() - t
                t = time.time()
                    
                acc = acc_value.eval(feed_dict={board: holdout_features, outcome: holdout_targets}).mean()
                print "Batch {} Hold-Out Accuracy: {} Time taken: {:.1f}s".format(i, acc, delta_t)

                summary = sess.run(all_summaries, feed_dict={board: holdout_features, outcome: holdout_targets})
                train_writer.add_summary(summary, i)            
            
            batch = get_batch(batch_size, i, [features, targets], how=batch_how)        
            train_step.run(feed_dict={board: batch[0], outcome: batch[1]})
        
        print "DONE TRAINING"
        print "FINAL ACCURACY: {}".format(acc)
        train_writer.close()
        
        if save_prefix:
            save_name = "{}_{}".format(save_prefix, suffix)
            print "SAVING TO: {}".format(save_name)
            os.makedirs(save_name)
            tf.train.Saver().save(sess, save_name+'/model')

## Attempt 1: Normal dense fully-conneected neural network

In [ ]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy
from keras.regularizers import l2

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board')
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')

    # Fully connected layers
    x = Dense(512, activation='tanh')(board)    
    x = Dense(512, activation='tanh')(x) 
    x = Dense(512, activation='tanh')(x)
    x = Dense(48, activation='tanh')(x)

    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
        
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)    
    
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [ ]:
run(graph, './train/dense', batch_size=250, num_batches=80000, save_prefix='./models/dense')

Running ./train/dense_2017_05_29_23:18:21
Batch 0 Hold-Out Accuracy: 0.344199985266 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.577839970589 Time taken: 15.9s
Batch 2000 Hold-Out Accuracy: 0.581120014191 Time taken: 16.6s
Batch 3000 Hold-Out Accuracy: 0.588999986649 Time taken: 16.6s
Batch 4000 Hold-Out Accuracy: 0.581700026989 Time taken: 16.7s
Batch 5000 Hold-Out Accuracy: 0.590020000935 Time taken: 16.8s
Batch 6000 Hold-Out Accuracy: 0.59636002779 Time taken: 17.3s
Batch 7000 Hold-Out Accuracy: 0.595139980316 Time taken: 16.9s
Batch 8000 Hold-Out Accuracy: 0.596759974957 Time taken: 16.9s
Batch 9000 Hold-Out Accuracy: 0.603739976883 Time taken: 17.0s
Batch 10000 Hold-Out Accuracy: 0.603219985962 Time taken: 16.9s
Batch 11000 Hold-Out Accuracy: 0.602540016174 Time taken: 17.1s
Batch 12000 Hold-Out Accuracy: 0.601279973984 Time taken: 16.9s
Batch 13000 Hold-Out Accuracy: 0.60492002964 Time taken: 17.2s
Batch 14000 Hold-Out Accuracy: 0.610700011253 Time taken: 16.9s
Batch 15000 Ho

## Attempt 2: A number of parallel CovNets

In [ ]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')
    
    d1 = Dense(512, activation='tanh')(board) 
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)    
    
    c1 = Flatten()(Conv2D(1, kernel_size=(4, 4), activation='relu', input_shape=(7, 6, 1))(rs))
    c2 = Flatten()(Conv2D(1, kernel_size=(1, 4), activation='relu', input_shape=(7, 6, 1))(rs))
    c3 = Flatten()(Conv2D(1, kernel_size=(4, 1), activation='relu', input_shape=(7, 6, 1))(rs))
    c4 = Flatten()(Conv2D(1, kernel_size=(5, 5), activation='relu', input_shape=(7, 6, 1))(rs))
    c5 = Flatten()(Conv2D(1, kernel_size=(3, 3), activation='relu', input_shape=(7, 6, 1))(rs))    
    c6 = Flatten()(Conv2D(1, kernel_size=(2, 2), activation='relu', input_shape=(7, 6, 1))(rs))

    merged = Concatenate()([d1, c1, c2, c3, c4, c5, c6])
    
    x = Dense(256, activation='tanh')(merged)         
    x = Dense(128, activation='tanh')(x)    
    x = Dense(12, activation='tanh')(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [ ]:
sess = run(graph, './train/conv2d', batch_size=250, num_batches=80000, save_prefix='./models/cov2d')